In [1]:
from googletrans import Translator
import numpy as np
import random
import pandas as pd
import time
import gezi
from gezi import tqdm
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from src import util

In [2]:
root = '../input/feedback-prize-2021'

In [3]:
df = pd.read_feather(f'{root}/train_en.fea')

In [4]:
df

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,worker
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones [SEP] [SEP],Phones [SEP] [SEP] Modern humans today are alw...,0,1,86
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone....,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,"When people have phones, they know about certa...",Phones [SEP] [SEP] Modern humans today are alw...,4,1,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,175155,4C471936CD75,18,10,2,576,828,582,834.0,seeking multiple opinions before making a hard...,"In ancient times, and also still today in some...",175155,1,91
175156,175156,4C471936CD75,18,11,3,828,839,834,845.0,it is better to seek multiple opinions instead...,"In ancient times, and also still today in some...",175156,1,91
175157,175157,4C471936CD75,18,12,2,839,903,845,911.0,The impact of asking people to help you make a...,"In ancient times, and also still today in some...",175157,1,91
175158,175158,4C471936CD75,18,13,0,903,905,911,913.0,"Of course,","In ancient times, and also still today in some...",175158,1,91


In [5]:
import re,string
regex = re.compile('[%s]' % re.escape(string.punctuation))
def remove_punct(s):  # From Vinko's solution, with fix.
  return regex.sub('', s)
s = "string. With. Punctuation"
remove_punct(s)

'string With Punctuation'

In [6]:
paras = []
id = None
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  para = row['para'].replace('[SEP]', '')
  para = remove_punct(para)
  paras.append(para)
df['para'] = paras
df.head(5)

  0%|          | 0/175160 [00:00<?, ?it/s]

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,worker
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones,Phones [SEP] [SEP] Modern humans today are alw...,0,1,86
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone ...,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,When people have phones they know about certai...,Phones [SEP] [SEP] Modern humans today are alw...,4,1,86


In [7]:
ids = []
texts = []
id = None
words = []
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  if row['id'] != id:
    if words:
      ids.append(id)
      texts.append(' '.join(words))
    words = []
    id = row['id']
  col = 'para'
  words.extend(str(row[col]).split(' '))
if words:
  ids.append(id)
  texts.append(' '.join(words))
d_text = pd.DataFrame({
  'id': ids,
  'text': texts
})
d_text

  0%|          | 0/175160 [00:00<?, ?it/s]

,id,text
0,423A1CA112E2,Phones Modern humans today are always on the...
1,A8445CABFECE,Phones Driving Drivers should not be able t...
2,6B4F7A0165B9,Cell Phone Operation While Driving The abili...
3,E05C7F5C1156,People are debating whether if drivers should ...
4,50B3435E475B,Texting and driving Over half of drivers in ...
...,...,...
15589,8F4B595CF9E7,Do you ever want more opinions and options whe...
15590,6B5809C83978,Has anyone ever gave you advice Was the advice...
15591,408A7D3D2EEC,Imagine seeking advice from multiple people an...
15592,AFEC37C2D43F,There has been at least one point in everyones...


In [8]:
df = pd.merge(df, d_text, on=['id'], how='inner', suffixes=('_ori', ''))

In [9]:
df.head(20)

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,worker,text
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones,Phones [SEP] [SEP] Modern humans today are alw...,0,1,86,Phones Modern humans today are always on the...
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone ...,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86,Phones Modern humans today are always on the...
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86,Phones Modern humans today are always on the...
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86,Phones Modern humans today are always on the...
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,When people have phones they know about certai...,Phones [SEP] [SEP] Modern humans today are alw...,4,1,86,Phones Modern humans today are always on the...
5,5,423A1CA112E2,17,5,1,139,163,145,169.0,Driving is one of the way how to get around Pe...,Phones [SEP] [SEP] Modern humans today are alw...,5,1,86,Phones Modern humans today are always on the...
6,6,423A1CA112E2,17,6,2,163,211,169,219.0,Thats why theres a thing thats called no texti...,Phones [SEP] [SEP] Modern humans today are alw...,6,1,86,Phones Modern humans today are always on the...
7,7,423A1CA112E2,17,7,2,211,282,219,294.0,Sometimes on the news there is either an accid...,Phones [SEP] [SEP] Modern humans today are alw...,7,1,86,Phones Modern humans today are always on the...
8,8,423A1CA112E2,17,8,1,282,297,294,309.0,Phones are fine to use and its also the best w...,Phones [SEP] [SEP] Modern humans today are alw...,8,1,86,Phones Modern humans today are always on the...
9,9,423A1CA112E2,17,9,2,297,355,309,367.0,If you go through a problem and you cant find ...,Phones [SEP] [SEP] Modern humans today are alw...,9,1,86,Phones Modern humans today are always on the...


In [10]:
df[df.id =='423A1CA112E2']

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,worker,text
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones,Phones [SEP] [SEP] Modern humans today are alw...,0,1,86,Phones Modern humans today are always on the...
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone ...,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86,Phones Modern humans today are always on the...
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86,Phones Modern humans today are always on the...
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86,Phones Modern humans today are always on the...
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,When people have phones they know about certai...,Phones [SEP] [SEP] Modern humans today are alw...,4,1,86,Phones Modern humans today are always on the...
5,5,423A1CA112E2,17,5,1,139,163,145,169.0,Driving is one of the way how to get around Pe...,Phones [SEP] [SEP] Modern humans today are alw...,5,1,86,Phones Modern humans today are always on the...
6,6,423A1CA112E2,17,6,2,163,211,169,219.0,Thats why theres a thing thats called no texti...,Phones [SEP] [SEP] Modern humans today are alw...,6,1,86,Phones Modern humans today are always on the...
7,7,423A1CA112E2,17,7,2,211,282,219,294.0,Sometimes on the news there is either an accid...,Phones [SEP] [SEP] Modern humans today are alw...,7,1,86,Phones Modern humans today are always on the...
8,8,423A1CA112E2,17,8,1,282,297,294,309.0,Phones are fine to use and its also the best w...,Phones [SEP] [SEP] Modern humans today are alw...,8,1,86,Phones Modern humans today are always on the...
9,9,423A1CA112E2,17,9,2,297,355,309,367.0,If you go through a problem and you cant find ...,Phones [SEP] [SEP] Modern humans today are alw...,9,1,86,Phones Modern humans today are always on the...


In [11]:
starts, ends = [], []
end = 0
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  if row['para_id'] == 0:
    end = 0
  start = end
  col = 'para' 
  end = start + len(str(row[col]).replace('[SEP]', '\n').split())
  starts.append(start)
  ends.append(end)
df['start'] = starts
df['end'] = ends

  0%|          | 0/175160 [00:00<?, ?it/s]

In [12]:
df

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,worker,text
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones,Phones [SEP] [SEP] Modern humans today are alw...,0,1,86,Phones Modern humans today are always on the...
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone ...,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86,Phones Modern humans today are always on the...
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86,Phones Modern humans today are always on the...
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86,Phones Modern humans today are always on the...
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,When people have phones they know about certai...,Phones [SEP] [SEP] Modern humans today are alw...,4,1,86,Phones Modern humans today are always on the...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,175155,4C471936CD75,18,10,2,576,828,582,834.0,seeking multiple opinions before making a hard...,"In ancient times, and also still today in some...",175155,1,91,In ancient times and also still today in some ...
175156,175156,4C471936CD75,18,11,3,828,839,834,845.0,it is better to seek multiple opinions instead...,"In ancient times, and also still today in some...",175156,1,91,In ancient times and also still today in some ...
175157,175157,4C471936CD75,18,12,2,839,903,845,911.0,The impact of asking people to help you make a...,"In ancient times, and also still today in some...",175157,1,91,In ancient times and also still today in some ...
175158,175158,4C471936CD75,18,13,0,903,905,911,913.0,Of course,"In ancient times, and also still today in some...",175158,1,91,In ancient times and also still today in some ...


In [13]:
df[['id', 'para_type', 'start', 'end', 'text', 'kfold', 'worker']].reset_index().to_feather(f'{root}/train_nopunct.fea')

In [14]:
df[df.id =='423A1CA112E2']

,index,id,part,para_id,para_type,start,end,start2,end2,para,text_ori,pid,kfold,worker,text
0,0,423A1CA112E2,17,0,0,0,1,0,3.0,Phones,Phones [SEP] [SEP] Modern humans today are alw...,0,1,86,Phones Modern humans today are always on the...
1,1,423A1CA112E2,17,1,5,1,45,3,47.0,Modern humans today are always on their phone ...,Phones [SEP] [SEP] Modern humans today are alw...,1,1,86,Phones Modern humans today are always on the...
2,2,423A1CA112E2,17,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [SEP] [SEP] Modern humans today are alw...,2,1,86,Phones Modern humans today are always on the...
3,3,423A1CA112E2,17,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [SEP] [SEP] Modern humans today are alw...,3,1,86,Phones Modern humans today are always on the...
4,4,423A1CA112E2,17,4,2,76,139,80,145.0,When people have phones they know about certai...,Phones [SEP] [SEP] Modern humans today are alw...,4,1,86,Phones Modern humans today are always on the...
5,5,423A1CA112E2,17,5,1,139,163,145,169.0,Driving is one of the way how to get around Pe...,Phones [SEP] [SEP] Modern humans today are alw...,5,1,86,Phones Modern humans today are always on the...
6,6,423A1CA112E2,17,6,2,163,211,169,219.0,Thats why theres a thing thats called no texti...,Phones [SEP] [SEP] Modern humans today are alw...,6,1,86,Phones Modern humans today are always on the...
7,7,423A1CA112E2,17,7,2,211,282,219,294.0,Sometimes on the news there is either an accid...,Phones [SEP] [SEP] Modern humans today are alw...,7,1,86,Phones Modern humans today are always on the...
8,8,423A1CA112E2,17,8,1,282,297,294,309.0,Phones are fine to use and its also the best w...,Phones [SEP] [SEP] Modern humans today are alw...,8,1,86,Phones Modern humans today are always on the...
9,9,423A1CA112E2,17,9,2,297,355,309,367.0,If you go through a problem and you cant find ...,Phones [SEP] [SEP] Modern humans today are alw...,9,1,86,Phones Modern humans today are always on the...
